# Benchmark for big data ingestion and querying

## Introduction

This notebook provides a quick comparison of the ingestion and querying speed between Shapelets (shapelets-platform) and two well known python libraries for tabular data handling: Pyarrow, Pandas and Polars. You will need to have them installed in order to run this benchmark.

In [ ]:
! pip install pandas pyarrow polars shapelets-platform

The benchmark is based on the NYC Taxi Dataset, with a size of ~37 Gb. The dataset contains more than 1.5 billion records of taxi trips in the area of New York City during 10 years (2009 to 2019). The dataset consists of .parquet files, one for each month of data.

We will be comparing two scenarios: one month of data (~430 Mb) and 1 year of data (~4.6 Gb). The benchmark evaluates two objectives:
- Data ingestion 
- Data querying with aggregation, in order to compute the average number of passengers for each day and month.

Please note both the execution times, CPU and memory used by each library, as for instance, Shapelets implementation is the fastest, uses all CPU cores available and does not require loading the data into memory.

In order to handle data, Shapelets relies very efficient data structures which rely on a technique named bitmap indexing, implemented in C++. This technique offers particularly good results in huge databases, providing faster retrieval of records and greater efficiency in insert, delete and update operations.

When handling time series, temporal indices are discretized and codified as bitmap indices, speeding up operations and providing several advantages, like the ability to store time series sampled at an arbitrarily high frequency.

**Note**: if you find a better implementation for Pandas, Pyarrow or Polars, feel free to raise an issue in this repo or e-mail us at hello@shapelets.io

## One Month Scenario
### Pandas


In [1]:
import pandas as pd 

In [2]:
df = pd.read_parquet('../Benchmarks/nyc-taxi/2009/01')

df['pickup_at'] = pd.to_datetime(df['pickup_at'])
df = df.set_index('pickup_at')

df.groupby([df.index.date,df.index.hour])['passenger_count'].mean()

            pickup_at
2009-01-01  0            1.714221
            1            1.723661
            2            1.679692
            3            1.644623
            4            1.566899
                           ...   
2009-01-31  19           1.834419
            20           1.841117
            21           1.874803
            22           1.901640
            23           1.937456
Name: passenger_count, Length: 744, dtype: float64

### Pandas with PyArrow engine

In [3]:
import pandas as pd

In [4]:
df = pd.read_parquet('../Benchmarks/nyc-taxi/2009/01', engine='pyarrow')

df['pickup_at'] = pd.to_datetime(df['pickup_at'])
df = df.set_index('pickup_at')

df.groupby([df.index.date,df.index.hour])['passenger_count'].mean()

            pickup_at
2009-01-01  0            1.714221
            1            1.723661
            2            1.679692
            3            1.644623
            4            1.566899
                           ...   
2009-01-31  19           1.834419
            20           1.841117
            21           1.874803
            22           1.901640
            23           1.937456
Name: passenger_count, Length: 744, dtype: float64

### Polars

In [11]:
import polars as pl

In [12]:
data = pl.scan_parquet('../Benchmarks/nyc-taxi/2009/01/*.parquet')

data.group_by(
        [
            pl.col("pickup_at").cast(pl.Date).alias("pickup_at_date"),
            pl.col("pickup_at").dt.hour().alias("pickup_at_hour"),
        ]
    ).agg(pl.mean("passenger_count")).collect()

pickup_at_date,pickup_at_hour,passenger_count
date,i8,f64
2009-01-24,15,1.767491
2009-01-04,10,1.750357
2009-01-29,9,1.529461
2009-01-26,8,1.564938
2009-01-17,23,1.949876
…,…,…
2009-01-05,22,1.726516
2009-01-10,21,1.885208
2009-01-16,22,1.850057


### Shapelets

In [9]:
from shapelets.data import sandbox

In [10]:
playground = sandbox()

taxis = playground.from_parquet("taxis", ["../Benchmarks/nyc-taxi/2009/01/*.parquet"])

result = playground.from_sql("""
    SELECT
        CAST(pickup_at as DATE) as pickup_at_date,
        EXTRACT('hour' from pickup_at) as pickup_at_hour,                      
        AVG(passenger_count) as passenger_count                                                   
    FROM taxis
    GROUP BY 
        pickup_at_date, pickup_at_hour                                                                           
""").execute()

result.to_pandas()

,pickup_at_date,pickup_at_hour,passenger_count
0,2009-01-01,0,1.714221
1,2009-01-01,1,1.723661
2,2009-01-01,2,1.679692
3,2009-01-01,3,1.644623
4,2009-01-01,4,1.566899
...,...,...,...
739,2009-01-31,19,1.834419
740,2009-01-31,20,1.841117
741,2009-01-31,21,1.874803
742,2009-01-31,22,1.901640


## One Year Scenario

### Pandas (Large memory consumption - Do not run)

In [1]:
import pandas as pd 

df = pd.read_parquet('../Benchmarks/nyc-taxi/2009')

df['pickup_at'] = pd.to_datetime(df['pickup_at'])
df = df.set_index('pickup_at')

df.groupby([df.index.date,df.index.hour])['passenger_count'].mean()

: 

### Polars (Large memory consumption)

In [5]:
import polars as pl

In [6]:
data = pl.scan_parquet('../Benchmarks/nyc-taxi/2009/**/*.parquet')

data.group_by(
        [
            pl.col("pickup_at").cast(pl.Date).alias("pickup_at_date"),
            pl.col("pickup_at").dt.hour().alias("pickup_at_hour"),
        ]
    ).agg(pl.mean("passenger_count")).collect()

pickup_at_date,pickup_at_hour,passenger_count
date,i8,f64
2009-12-12,4,1.772643
2009-12-01,18,1.66098
2009-05-16,0,1.866671
2009-11-06,17,1.693558
2009-09-13,16,1.740269
…,…,…
2009-03-04,21,1.666816
2009-11-14,15,1.800349
2009-01-06,10,1.598939


### Shapelets

In [7]:
from shapelets.data import sandbox

In [8]:
playground = sandbox()

taxis = playground.from_parquet("taxis", ["../Benchmarks/nyc-taxi/2009/**/*.parquet"])

result = playground.from_sql("""
    SELECT
        CAST(pickup_at as DATE) as pickup_at_date,
        EXTRACT('hour' from pickup_at) as pickup_at_hour,                      
        AVG(passenger_count) as passenger_count                                                   
    FROM taxis
    GROUP BY 
        pickup_at_date, pickup_at_hour                                                                           
""").execute()

result.to_pandas()

,pickup_at_date,pickup_at_hour,passenger_count
0,2009-01-21,7,1.551747
1,2009-01-25,12,1.739060
2,2009-01-21,13,1.624000
3,2009-01-21,19,1.650012
4,2009-01-22,3,1.632184
...,...,...,...
8755,2009-12-20,4,1.781036
8756,2009-12-25,6,1.632369
8757,2009-12-13,6,1.655344
8758,2009-12-06,3,2.166596
